In [ ]:
In which I use extremely randomized trees. See

In [2]:
#Import libraries:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
#load data:
# train = pd.read_csv("train.csv")
# target = train['target']
# #drop targets & (unique row) IDs from training data
# train = train.drop(['ID','target'],axis=1)
# test = pd.read_csv("test.csv")
# IDs = test['ID'].values
# test = test.drop(['ID'],axis=1)

# PREPROCESSING

In [3]:
#https://www.kaggle.com/director/bnp-paribas-cardif-claims-management/simple-xgboost-0-46146/code
train = pd.read_csv("train.csv")
target = train['target']
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
ids = test['ID'].values
test = test.drop(['ID'],axis=1)
#

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()  #TODO

In [17]:
# https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19133/feature-engineering-for-beginners
#check this out: https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19240/analysis-of-duplicate-variables-correlated-variables-large-post

A function to report best hyperparameters:
http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

In [4]:
from operator import itemgetter
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

# TESTING EXTRATREES CLASSIFIER APPROACH

In [102]:
#https://www.kaggle.com/yuhaichina/bnp-paribas-cardif-claims-management/extratreesclassifier-score-0-45911
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

t0 = time.time()
X_train = train
X_test = test
extc = ExtraTreesClassifier(n_estimators=700,max_features= 50,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf= 5)      

extc.fit(X_train,target) 
preds = extc.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time

KeyboardInterrupt: 

In [105]:
import csv
predictions_file = open("extc.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(IDs, preds[:,1]))
predictions_file.close()

# RandomizedSearchCV on ETC parameter space

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import metrics

t0 = time.time()
X_train = train
X_test = test

param_grid = {'max_depth': range(20,50),
                      'n_estimators': np.arange(500,1000,100),
#                         'n_estimators': [10],
                      'max_features' : np.arange(10,110,10),
                      'min_samples_split' : np.arange(1,6,1),
                        'min_samples_leaf' : np.arange(1,6,1),
                      'criterion' : ['entropy'],
                      #'scale_pos_weight': [0.5, 1]
                      #'model__eta':[0.01,0.02],
                     #'model__scale_pos_weight':[0.8,1.0]
                      #'model__silent':[1],
                      }



extc = ExtraTreesClassifier()

n_iter_search=50
random_search = RandomizedSearchCV(extc, param_distributions=param_grid,
                                   n_iter=n_iter_search, scoring ="log_loss")

random_search.fit( train , target)

t1 = time.time()
total_time = t1 - t0
print total_time

83586.8007469


/Users/hugobowne-anderson/repos/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hugobowne-anderson/repos/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [23]:
print report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: -0.460 (std: 0.001)
Parameters: {'min_samples_leaf': 2, 'n_estimators': 800, 'max_features': 60, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': 29}

Model with rank: 2
Mean validation score: -0.460 (std: 0.001)
Parameters: {'min_samples_leaf': 2, 'n_estimators': 800, 'max_features': 50, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': 29}

Model with rank: 3
Mean validation score: -0.461 (std: 0.001)
Parameters: {'min_samples_leaf': 2, 'n_estimators': 800, 'max_features': 60, 'criterion': 'entropy', 'min_samples_split': 1, 'max_depth': 34}

None


In [16]:
params = {'min_samples_leaf': 2, 
         'n_estimators': 800, 
         'max_features': 50, 
         'criterion': 'entropy', 
         'min_samples_split': 2, 
         'max_depth': 29}
t0 = time.time()
extc2 = ExtraTreesClassifier(**params)
#OR
#extc2 = ExtraTreesClassifier(**random_search.best_params_)
extc2.fit(X_train,target) 
preds_2 = extc.predict_proba(X_test)[:,1]
t1 = time.time()
total_time = t1 - t0
print total_time

1220.19769096


In [18]:
import csv
predictions_file = open("extc2_rcv.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(ids, preds))
predictions_file.close()

This gave -logloss = 0.45450.

In [20]:
feat_import = extc.feature_importances_

In [22]:
print np.shape(feat_import)
print feat_import

(131,)
[ 0.00549517  0.00527297  0.00093389  0.0045345   0.00486351  0.00575246
  0.00474514  0.00353734  0.00519375  0.02164855  0.00479493  0.01975782
  0.00427068  0.01722462  0.00458783  0.00546969  0.00378255  0.0050976
  0.00424407  0.00463785  0.01541078  0.01620103  0.0044904   0.01816264
  0.00352862  0.00460647  0.00477159  0.00576614  0.00398048  0.01236534
  0.01865031  0.00445256  0.00408777  0.01691225  0.00497966  0.00527311
  0.00529791  0.00175697  0.00459593  0.01872522  0.00409869  0.00485325
  0.00451497  0.00488089  0.00528346  0.00344552  0.01041913  0.00384544
  0.00440344  0.10211347  0.00444687  0.01888435  0.00488473  0.00369635
  0.00444121  0.01714748  0.0054613   0.00522127  0.00468981  0.00457418
  0.00436079  0.01152769  0.00350043  0.0037437   0.0041299   0.03394047
  0.00418748  0.00488513  0.00559634  0.00525833  0.00892003  0.00948029
  0.00437829  0.00108883  0.00907405  0.00379577  0.00432269  0.00554185
  0.01098446  0.00505551  0.00485196  0.00549